# Neural networks

## Overview and terms

As we saw from the logistic regression yesterday, linear classifiers are often not the best at solving complicated problems. Neural networks introduce nonlinearity. They were originally developed as mathematical models of the information processing capabilities of biological brains, and were popular in the 80s and early 90s. Recently they have become popular again, especially as deep neural networks DNNs, including convolutional NNs (CNN), recurrent NNs (RNN), etc. 

<!---Those are beyond the scope of this class, but we will introduce the basics of NNs.-->

Below is a diagram of a simple NN (a MLP, multilayer perceptron, with a single hidden layer):
![NNFig](https://upload.wikimedia.org/wikipedia/commons/4/46/Colored_neural_network.svg)

A NN is made up of _neurons_ that receive multiple inputs, process them, and send the output to the neurons in the following layer. Here, the network is _dense_; every node of the hidden layer is connected to all previous and following nodes. 
The arrows indicate that information flows from left (the input) to the right (the output). Every arrow has an assigned weight that is a free parameter in the training of the network. 

Computing a series of weighted sums as is done by the above network is mathematically the same as computing just one weighted sum. An MLP with multiple linear hidden layers that all just sum up the inputs is thus equivalent to an MLP with a single linear hidden layer. To make this model more powerful than a linear model, a nonlinear function, the _activation function_, is applied to the weighted sum for each neuron in the hidden layer and used to determine the output that is propagated as input to the following neurons.

The number of neurons in the output layer and the choice of _output activation function_ depend on the task the network is intended for. For binary classification tasks, a typical setup has a single output neuron with a logistic sigmoid activation.
Large neural networks made up of many hidden layers of computation go under the term _deep learning_. Note that for dense layers, the number of parameters increases linearly in the number of hidden layers but quadratically in the number of neurons in each layer.

In order to train the NN, we have to determine the weight matrices $\Theta^{(i)}$ (which are basically just a bookkeeping devices holding the weights of all input connections for all neurons of one layer each) that minimize the _cost function_. 
This is done using a method called _backpropagation_.
The cost function of a NN is similar to what we have for logistic regression, modified to take into account possible multiple outputs, and with more complicated regularization.

## Basics of the mathematics behind NNs
* One can write a neuron's output as $a_i = g\left(\theta_i^T x_i\right)$, where $i$ is the index of the neuron, $x_i$ its vector of inputs, and $\theta_i$ are the weights of the input connections of the neuron (also a vector). $g$ is the activation function. 
* The NN above has an input layer (layer 1), a single hidden layer (layer 2), and an output layer (layer 3). One can have more hidden layers. 
* Combining the outputs of all neurons from one layer into one vector $z$, we can write $z^{(j)} = \Theta^{(j-1)}a^{(j-1)}$ with an upper index to label the layer. The matrix $\Theta^{(j-1)}$ is formed from all weights of all neurons in layer $j-1$.
* Then $a^{(j)} = g(z^{(j)})$. Thus evaluating the NN is a series of matrix multiplications followed by activation functions. (Which makes obvious that without the nonlinearity introduced by the activation functions, the whole NN would still be a linear map.)

### Backpropagation
![Neuron][2]

[2]: https://upload.wikimedia.org/wikipedia/commons/thumb/6/60/ArtificialNeuronModel_english.png/640px-ArtificialNeuronModel_english.png "a single neuron of an artifical network"

Here: $\text{net}_j = \sum_i w_{ij} x_i + b_j$ (the bias $b_j$ is not shown).

To train the network, the cost function $E$ is minimized by propagating "errors" in the outputs $\text{net}_j$ <!--- $z_i^{(j)}$ --> backwards. 
The partial derivatives of the cost function with respect to the weights can be obtained from the chain rule,
$$\frac{\partial E}{\partial w_{ij}} 
    = \frac{\partial E}{\partial o_j} \frac{\partial o_j}{\partial\text{net}_j} \frac{\partial \text{net}_j}{\partial w_{ij}}$$ 
and the weights are modified such that the cost function is reduced:
$$ \Delta w_{ij} = - \eta \frac{\partial E}{\partial w_{ij}}.$$ <!--= - \eta \delta_j o_i -->
$\eta$ is a metaparameter in the training that determines the _learning rate_.

In the derivative of the cost function, the terms from right to left are
* the output of the neuron $i$ feeding into the neuron $j$ with weight $w_{ij}$, $\frac{\partial \text{net}_j}{\partial w_{ij}} = x_i$,
* the derivative of the activation function, $\frac{\partial o_j}{\partial\text{net}_j} = \frac{\partial \varphi}{\partial\text{net}_j} = \varphi'(\text{net}_j)$
* and for a neuron in the last layer, $\frac{\partial E}{\partial o_j}$ is per definition the derivative of the cost function with respect to the output of this neuron.

Using the quadratic difference $E = (\hat y - y)^2$ as the cost function, for a neuron in the output layer:
$$\frac{\partial E}{\partial w_{ij}} 
    = 2(\hat y - y) \varphi'(\text{net}_j) x_i =: \delta_j x_i$$ 

For a neuron in a hidden layer $\frac{\partial E}{\partial o_j}$ depends on all the following layers, which is why we need to compute backwards. <!--- Here, the notation becomes impossible if we don't use an additional index for the layer -->

For more details on backpropagation, look, for example, at the [ML course][1] mentioned above.
The backpropagation formula is explicitly written down in a clean way, e.g., in [this wiki][2].
Another explanation of the backpropagation with lots of nice animations is given by 3Blue1Brown in the [video "Backpropagation calculus"][3] as part of their series on Deep learning.


[1]: https://www.coursera.org/learn/machine-learning
[2]: https://brilliant.org/wiki/backpropagation/
[3]: https://www.youtube.com/watch?v=tIeHLnjs5U8

### Minimization

Although the loss function of a neural network is typically highly non-convex with many local minima the optimization is typically performed via gradient descent. That means the parameters are updated step-by-step by following the gradient (steepest descent/ascent). The total cost function of the whole training dataset is the mean of the cost of each individual training example. This allows to perform a **stochastic gradient descent** (SGD) by calculating gradient updates only on a random subset (**batch**) of the training data. The advantages of this method are:

* less computational effort for each gradient update since only a subset of examples has to be (back-)propagated through the network
* less memory consumption
* random fluctuations can help to escape local minima

On the other hand, typically more gradient steps are needed when the gradient is calculated from smaller batches and too small batch sizes can lead to large fluctuations of the loss value during training. As a consequence there is a trade-off between fast computation of each gradient step and the total number of gradient steps needed that is tuned by choosing the appropriate batch size. There are many improvements to the plain gradient descent that try to adjust the step sizes (**learning rate**) dynamically, possibly on a per-parameter-base. One of the most popular optimization algorithm currently (2019) is [Adam](https://arxiv.org/abs/1412.6980v8). A nice overview can be found at http://ruder.io/optimizing-gradient-descent/index.html or https://arxiv.org/abs/1609.04747.

### Activation functions
The activation function of a neuron transforms the net input into the activation (output) of the neuron, i.e. whether the neutron is firing or not.

Popular choices include ([source][2]):
![1]

* For example, if we use a logistic function as the activation function, we can have $g\left(\theta^Tx\right) = \frac{1}{1+\mathrm{exp}\left(-\theta^Tx\right)}$, 
* or if a Rectified Linear Unit (ReLU), $g\left(\theta^Tx\right) = \mathrm{max}\left(0, \theta^Tx\right)$. 

[1]: figures/activation_functions.png "overview of commonly used activation functions"
[2]: https://stanford.edu/~shervine/teaching/cs-229/cheatsheet-deep-learning

* [Keras][4] provides a large set of activation functions: sigmoid, tanh, relu (rectified linear unit), linear, [elu][3] (exponential linear unit), [softmax][5], ...
* In addition, e.g. learnable activations (which maintain a state) are available as advanced activation layers. These include `PReLU` and `LeakyReLU`.
* Differentiable functions allow the network to be trained with gradient descent.
* Activation functions map the (potentially unbounded) input range of the net input of the neurons to a finite output range and are therefore sometimes  referred to as squashing functions.

[3]: https://arxiv.org/pdf/1511.07289.pdf
[4]: https://keras.io/activations/
[5]: https://de.wikipedia.org/wiki/Softmax-Funktion